In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  

from selenium.webdriver.support.ui import Select

#1. 크롤링 기본 셋팅
print("=" *80)
print("연습문제 6-4 : 나라장터 크롤링하기")
print("=" *80)
query_txt = '나라장터'


search_key = input('1. 공고명으로 검색할 키워드는 무엇입니까?: ')
start_date = input('2. 조회 시작일자 입력(예:2019/01/01):')
end_date = input('3. 조회 종료일자 입력(예:2019/03/31):')
f_dir = input("4.파일로 저장할 폴더 이름을 쓰세요(예:/Users/kibeomkim/Desktop/folder1/): ")
if f_dir=='' :
    f_dir='/Users/kibeomkim/Desktop/웹크롤링_결과_데이터/'

    
# 저장될 파일위치와 이름 지정하기
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name= s+'-'+query_txt+'.txt'
fc_name= s+'-'+query_txt+'.csv'
fx_name= s+'-'+query_txt+'.xls'

#크롬드라이버 불러오기 
s_time = time.time()

path = "/Users/kibeomkim/Desktop/chromedriver_240/chromedriver"
driver = webdriver.Chrome(path)

#나라장터 '검색' 페이지로 이동 - <크롤링 시작>
driver.get('http://www.g2b.go.kr/index.jsp')
time.sleep(random.randrange(2,5))

#1. 공고명 검색창 찾고, 검색 키워드 넣기 
search_option = driver.find_element_by_name("bidNm")
search_option.send_keys(search_key)

#2. 조회 시작일자 입력
date_option0 = driver.find_element_by_id('fromBidDt')
date_option0.send_keys('\b'*11)
date_option0.send_keys(start_date)

#3. 조회 종료일자 입력
date_option2 = driver.find_element_by_id("toBidDt")
date_option2.send_keys('\b'*11)
date_option2.send_keys(end_date)

#4. 검색 버튼 클릭
driver.find_element_by_class_name('btn_dark').click()
time.sleep(3) #페이지 전환 3초간 대기 

#내가 원하는 아이프레임 속으로 들어가서 html 코드 가져오기
asdf = driver.switch_to.frame('sub')
html900 = driver.page_source
soup900 = BeautifulSoup(html900, 'html.parser')
real_url = soup900.find('frame', title='콘텐츠프레임')['src']

driver.close()
path = "/Users/kibeomkim/Desktop/chromedriver_240/chromedriver"
driver = webdriver.Chrome(path)
driver.get(real_url)
time.sleep(2)


sort01 = []
number01 = []
sorted11 = []
name01 = []
url01 = []
insti01 = []
prov11 = []
howto11 = []
input_date01 = []
com01 = []
throw01 = []

#1페이지 도착 
#하지만 iframe을 깨야 진짜 정보에 접근할 수 있다. 
#프레임 속으로 한단계 더 들어가자

#프레임 깨기
#html3 = driver.page_source 
#soup3 = BeautifulSoup(html3, 'html.parser')

#real_url = soup3.find('frame', id ='sub').find('frame', name_='main')
#r2 = real_url['src']
#driver.get(r2)
#time.sleep(3)



no1 = 0

no2 = 2

f = open(ff_name, 'a', encoding='utf-8')

while no2 == 2 : 
    html0 = driver.page_source 
    soup0 = BeautifulSoup(html0, 'html.parser')
    contents_list = soup0.find('table', 'table_list_tbidTbl table_list').find('tbody').find_all('tr')
    
    for a in contents_list :
        no1 +=1
        contents_list2 = a.find_all('td')
        f.write('%s번째 공고내용을 추출합니다=================' %no1)
        f.write('\n')
    
        sort = contents_list2[0].get_text().strip()
        sort01.append(sort)
        f.write('1. 업무 : '+sort)
        f.write('\n')
    
        number = contents_list2[1].get_text().replace('\n', "").strip()
        number01.append(number)
        f.write('2. 공고번호-차수 : '+number)
        f.write('\n')
    
        sorted1 = contents_list2[2].get_text().strip()
        sorted11.append(sorted1)
        f.write('3. 분류 : '+sorted1)
        f.write('\n')
    
        name = contents_list2[3].get_text().replace('\n', "").strip()
        name01.append(name)
        f.write('4. 공고명 : '+name)
        f.write('\n')
    
        url = contents_list2[3].find('a')['href'].replace('\n', "").strip()
        url01.append(url)
        f.write('5. URL 주소 : '+url)
        f.write('\n')
    
        insti = contents_list2[4].get_text().strip()
        insti01.append(insti)
        f.write('6. 공고기관 : '+insti)
        f.write('\n')
    
        prov = contents_list2[5].get_text().strip()
        prov11.append(prov)
        f.write('7. 수요기관 : '+prov)
        f.write('\n')
    
        howto = contents_list2[6].get_text().strip()
        howto11.append(howto)
        f.write('8. 계약방법 : '+howto)
        f.write('\n')
    
        input_date = contents_list2[7].get_text().replace('\n', "").replace('"', "").strip()
        input_date01.append(input_date)
        f.write('9. 입력일시(입찰마감일시) : '+input_date)
        f.write('\n')
    
        try : 
            com = contents_list2[8].get_text().strip()
            com01.append(com)
            f.write('10. 공동수급 : '+com)
            f.write('\n')
        except : 
            print("")
            f.write('10. 공동수급 : ')
            f.write('\n')
        
        try : 
            throw = contents_list2[9].get_text().strip()
            throw01.append(throw)
            f.write('11. 투찰여부 : '+throw)
            f.write('\n')
            f.write('\n')
        except : 
            print("")
            f.write('11. 투찰여부 : ')
            f.write('\n')
            f.write('\n')

    



#페이지 넘어가기
    try : 
        driver.find_element_by_link_text('+ 더보기').click()    
        time.sleep(3)
    except : 
        break

print('총 추출 완료한 건수는 %s 건 입니다===============================' %no1)
f.close()
driver.close()

e_time = time.time()

total_time = e_time - s_time 


#csv, xlsx 파일로 저장하기 
korea = pd.DataFrame()

korea['업무'] = sort01
korea['공고번호-차수'] = number01
korea['분류'] = sorted11
korea['공고명'] = name01
korea['공고URL주소'] = url01
korea['공고기관'] = insti01 
korea['수요기관'] = prov11
korea['계약방법'] = howto11
korea['입력일시(입찰마감일시)'] = input_date01
korea['공동수급'] = com01
korea['투찰'] = throw01

korea.to_csv(fc_name, encoding='utf-8-sig')
korea.to_excel(fx_name)

print('='*100)
print('총 소요시간은 %s 초 입니다' %total_time)
print('총 추출완료 건수는 %s 건 입니다' %no1)
print('파일 저장 완료 : csv 파일명 :'+f_dir+s+'-'+query_txt+fc_name)
print('파일 저장 완료 : xls 파일명 :'+f_dir+s+'-'+query_txt+fx_name)
print('='*100)


연습문제 6-4 : 나라장터 크롤링하기
1. 공고명으로 검색할 키워드는 무엇입니까?: 캠프
2. 조회 시작일자 입력(예:2019/01/01):2017.01.01
3. 조회 종료일자 입력(예:2019/03/31):2017.12.31
4.파일로 저장할 폴더 이름을 쓰세요(예:/Users/kibeomkim/Desktop/folder1/): 


KeyboardInterrupt: 

In [88]:
#프레임세트에서 html코드 찾는 방법 : 
asdf = driver.switch_to.frame('sub')

html90 = driver.page_source

NoSuchFrameException: Message: no such frame
  (Session info: chrome=87.0.4280.88)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.16.0 x86_64)


In [89]:
html90 = driver.page_source
soup90 = BeautifulSoup(html90, 'html.parser')
soup90

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Frameset//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-frameset.dtd">
<html lang="ko" xml:lang="ko" xmlns="http://www.w3.org/1999/xhtml"><head>
<title>입찰정보 하단프레임</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<script src="/gov/koneps/co/js/crossDomain.js" type="text/javascript"></script>
</head>
<frameset cols="175,*,0">
<frame frameborder="0" name="left" noresize="noresize" scrolling="auto" src="/pt/menu/leftMenu.do?ugroup=b" title="좌측메뉴프레임"/>
<frame frameborder="0" name="main" noresize="noresize" scrolling="auto" src="http://www.g2b.go.kr:8101/ep/tbid/tbidList.do?taskClCds=&amp;bidNm=%C4%B7%C7%C1&amp;searchDtType=1&amp;fromBidDt=2019/01/01&amp;toBidDt=2019/04/20&amp;fromOpenBidDt=&amp;toOpenBidDt=&amp;radOrgan=1&amp;instNm=&amp;area=&amp;regYn=Y&amp;bidSearchType=1&amp;searchType=1" title="콘텐츠프레임"/>
<frame frameborder="0" name="hframe" noresize="noresize" scrolling="no" src="about:blank" title="빈프레임"/>
</frames